In [3]:
import argparse
from Bio import AlignIO
from Bio.Align import AlignInfo
import numpy as np
from Bio import SeqIO
import pandas as pd
import glob

In [27]:
from Bio import AlignIO
from Bio.Align import AlignInfo
import pandas as pd

def calculate_conservation(msa_file):
    """
    Calculates the percentage of conservation of each residue from the first sequence in the alignment across all sequences in the MSA, excluding gaps.

    Parameters:
        msa_file (str): Path to the input MSA file (in FASTA format).

    Returns:
        pandas.DataFrame: DataFrame containing the position, conserved residue, and percentage conservation of each residue.
    """
    # Read the MSA file
    alignment = AlignIO.read(msa_file, "fasta")

    # Get the first sequence (original sequence)
    original_seq = str(alignment[0].seq)

    # Create an AlignInfo summary object
    summary = AlignInfo.SummaryInfo(alignment)

    # Get the length of the alignment
    alignment_length = alignment.get_alignment_length()

    # Initialize lists to store conservation values
    positions = []
    conserved_residues = []
    conservation_values = []

    # Iterate over each position in the alignment
    for pos in range(alignment_length):
        # Skip if the position in the original sequence is a gap
        if original_seq[pos] == "-":
            continue
        
        # Count occurrences of each residue at the current position
        residue_counts = {}
        for record in alignment:
            residue = record.seq[pos]
            if residue == "-":
                continue
            if residue in residue_counts:
                residue_counts[residue] += 1
            else:
                residue_counts[residue] = 1
        
        # Calculate conservation for the current position
        original_residue = original_seq[pos]
        if original_residue in residue_counts:
            conservation = residue_counts[original_residue] / len(alignment)
        else:
            conservation = 0
        
        # Add conserved residue information to lists
        positions.append(pos + 1)  # Convert to 1-based indexing
        conserved_residues.append(original_residue)
        conservation_values.append(conservation)
    
    # Create DataFrame
    df_conservation = pd.DataFrame({'Position': positions, 'Conserved Residue': conserved_residues, 'Conservation': conservation_values})

    return df_conservation


In [7]:
al="results/test_clu_seq0_og_aligned.fasta"
og="data/dummy_seq.fasta"

In [21]:
alignment = AlignIO.read(al, "fasta")
sum = AlignInfo.SummaryInfo(alignment)
aln_len = alignment.get_alignment_length()

In [29]:
#calculate_conservation(al)

### Extra - Conservation analysis : Trypsin

In [1]:
def create_dataframe(csv_file):
    """
    Read a CSV file into a pandas DataFrame.

    Parameters:
        csv_file (str): Path to the CSV file.

    Returns:
        pandas.DataFrame: DataFrame containing the data from the CSV file.
    """
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    return df

def load_csv_files(pattern):
    """
    Load CSV files matching the specified pattern into pandas DataFrames.

    Parameters:
        pattern (str): Pattern to match filenames.

    Returns:
        dict: Dictionary containing DataFrames, where keys are DataFrame names and values are DataFrames.
    """
    # Find all CSV files matching the pattern
    csv_files = glob.glob(pattern)

    # Dictionary to store DataFrames
    dfs = {}

    # Iterate over each CSV file and create a DataFrame
    for csv_file in csv_files:
        df_name = csv_file.split(".")[0]  # Extract DataFrame name from file name
        df = create_dataframe(csv_file)
        dfs[df_name] = df

    return dfs

In [17]:
# Example usage
pattern = "Tryp/results/tryp_clu_0_cs.csv"  # Pattern to match filenames
dfs = load_csv_files(pattern)


In [22]:
df0 = pd.read_csv(pattern, delimiter="\t")

In [32]:
sorted_df = df0.sort_values(by='Conservation', ascending=False)

In [33]:
sorted_df

,Conserved Residue,Conservation
185,G,0.928724
196,G,0.877471
155,C,0.858708
169,C,0.857575
187,P,0.848382
...,...,...
108,A,0.010326
122,E,0.008185
39,S,0.006800
0,F,0.006422
